In [1]:
import pandas as pd
import numpy as np

In [2]:
max_allocation = 100000 # maximum budget that can be allocated
weights = [0.30,0.20,0.15,0.15,0.10,0.10] # weights for n-month returns
mutation_probability = 0.5 # initial mutation probability
number_of_iterations = 25 # total number of GA iterations
number_of_chrom_in_population = 500 # size of population

In [3]:
scrip_names=['NVDA','AMZN','MU','AVGO','HD']
list_of_months = [3,6,12,18,24,36]

In [4]:
stck_returns= pd.read_csv("portopt2.csv")
stck_returns= pd.DataFrame(stck_returns)

stck_returns.set_index('Months')

,NVDA,AMZN,MU,AVGO,HD
Months,,,,,
3,0.705965,1.211964,0.414750,-0.038156,0.787997
6,0.395943,1.340490,1.514853,0.709030,0.671822
12,0.725880,1.281891,1.484242,0.714382,0.954735
18,2.139444,1.589002,1.630457,0.791490,1.023071
24,2.448978,0.733130,2.158251,0.053484,0.197019
36,3.755990,1.941009,1.381597,2.150344,0.915937


In [5]:
def gen_init_population(pop,num_stocks):
    init_population = pd.DataFrame(columns=scrip_names)
    for i in range(pop):
        random = np.random.random(num_stocks)
        chromosome = random/(sum(random))
        init_population.loc[i] = chromosome
    return init_population

In [6]:
def fitness_func(max_alloc,population,stock_ret,wts):
    weighted_stck_returns = stock_ret.apply(lambda x:(x*wts).sum())
    pfolio_return = population.apply(lambda x: sum((x*max_alloc)*weighted_stck_returns)/max_alloc, axis = 1)
    return pfolio_return

In [7]:

def selection_func(fit_func,max_alloc,population,stock_ret,wts):
    portfolio_return = fit_func(max_alloc,population,stock_ret,wts)
    selected_portfolio = portfolio_return.sort_values(ascending=False)[0:int(0.2*population.shape[0])]
    elite = population.iloc[selected_portfolio.index,:]
    return elite

In [8]:
tmp_pop = selection_func(fitness_func,max_allocation,gen_init_population(number_of_chrom_in_population,len(scrip_names)),stck_returns,weights)
tmp_pop.head()

C:\Users\Ganihar\Anaconda3\envs\wqu_ml_fin\lib\site-packages\pandas\compat\_optional.py:106: UserWarning: Pandas requires version '2.6.2' or newer of 'numexpr' (version '2.6.1' currently installed).
  warnings.warn(msg, UserWarning)


,NVDA,AMZN,MU,AVGO,HD
0,0.159193,0.091059,0.172098,0.287743,0.289907
1,0.227481,0.378809,0.160671,0.053773,0.179265
2,0.201417,0.148550,0.169969,0.230187,0.249876
3,0.333719,0.061319,0.211559,0.299492,0.093911
4,0.025096,0.334197,0.203227,0.261345,0.176135


In [9]:
def evolution_func(elite_pop,mut_prob,scrip,num_of_chrom_in_pop):
    new_pop = pd.DataFrame(columns=scrip)
    new_pop_iter = 0
    while(new_pop_iter <= num_of_chrom_in_pop):
        if np.random.random() < mut_prob:
            candidate = np.random.randint(low=0,high=elite_pop.shape[0],size=1)
            mutant = elite_pop.iloc[candidate,:].copy()
            np.random.shuffle(mutant)
            new_pop = new_pop.append(mutant,ignore_index=True)
            mut_prob = mut_prob/(new_pop_iter+1)
        else:
            parent_indices = np.random.randint(low=0,high=elite_pop.shape[0],size=2)
            parents = elite_pop.iloc[parent_indices,:]
            alpha = np.random.random()
            child1 = alpha*(parents.iloc[0,:]) + (1-alpha)*(parents.iloc[1,:])
            child2 = (1-alpha)*(parents.iloc[0,:]) + alpha*(parents.iloc[1,:])
            new_pop = new_pop.append(dict(child1),ignore_index=True)
            new_pop = new_pop.append(dict(child2),ignore_index=True)
        new_pop_iter+=1
    new_pop = pd.concat([new_pop,elite_pop],ignore_index=True)
    return new_pop

In [10]:
evolution_func(tmp_pop,mutation_probability,scrip_names,number_of_chrom_in_population).head()

,NVDA,AMZN,MU,AVGO,HD
0,0.349143,0.303116,0.194092,0.139938,0.013711
1,0.106769,0.066284,0.388362,0.294328,0.144257
2,0.006628,0.130059,0.142102,0.494231,0.226979
3,0.256657,0.110569,0.245186,0.172760,0.214829
4,0.237723,0.134951,0.266272,0.128971,0.232083


In [11]:
def main_func():
    initial_population = gen_init_population(number_of_chrom_in_population,len(scrip_names))
    elite_population = selection_func(fitness_func,max_allocation,initial_population,stck_returns,weights)
    elite_population.reset_index(inplace=True,drop=True)
    new_population = evolution_func(elite_population,mutation_probability,scrip_names,number_of_chrom_in_population)
    for i in range(number_of_iterations):
        new_elite_population = selection_func(fitness_func,max_allocation,new_population,stck_returns,weights)
        new_population = evolution_func(new_elite_population,mutation_probability,scrip_names,number_of_chrom_in_population)
    return new_population

In [12]:
range_of_portfolios = main_func()

In [13]:
range_of_portfolios

,NVDA,AMZN,MU,AVGO,HD
0,0.180674,0.196413,0.204961,0.207796,0.210156
1,0.181180,0.197711,0.204990,0.206891,0.209227
2,0.180828,0.197147,0.205022,0.207360,0.209643
3,0.180461,0.197805,0.204982,0.207344,0.209407
4,0.180840,0.198269,0.204403,0.206930,0.209558
...,...,...,...,...,...
1242,0.180985,0.197545,0.205277,0.206695,0.209499
1243,0.181561,0.197010,0.205080,0.207043,0.209305
1244,0.181840,0.197261,0.205683,0.206432,0.208784
1245,0.181705,0.197299,0.205688,0.206750,0.208558


In [14]:
top_portfolios = selection_func(fitness_func,max_allocation,range_of_portfolios,stck_returns,weights)
final_portfolio = top_portfolios.iloc[0]
final_portfolio

NVDA    0.180674
AMZN    0.196413
MU      0.204961
AVGO    0.207796
HD      0.210156
Name: 0, dtype: float64

In [15]:
weighted_stck_returns = stck_returns.apply(lambda x:(x*weights).sum())
weighted_stck_returns[1:6]

NVDA    1.341273
AMZN    1.329735
MU      1.248585
AVGO    0.576623
HD      0.778730
dtype: float64

In [18]:
portfolio_return = sum((final_portfolio*max_allocation)*weighted_stck_returns[1:6])/max_allocation
print('The portfolio return is: {0:.3f}'.format(portfolio_return))

The portfolio return is: 4.034
